In [9]:
'''
This code is provided to help with question 4.
This code returns optimal primal variables x
and dual variables y.
'''
import cvxpy as cp
import numpy as np
import numpy.linalg as la

def simplex_iteration(x, B, problem):
 """Perform one simplex iteration given
 - basic feasible solution x
 - basis B
 It returns new x, new basis, new dual variable,
 and termination flag (true/false)
 """
 A, b, c = problem['A'], problem['b'], problem['c']
 m, n = A.shape
 A_B = A[:, B]
 # Compute reduced cost vector
 p = la.solve(A_B.T, c[B])
 c_bar = c - A.T @ p
 # Check optimality
 if np.all(c_bar >= 0):
     print("Optimal solution found!")
     return x, B, -p, True

# Choose j such that c_bar < 0 (first one)
 j = np.where(c_bar < 0)[0][0]

# Compute search direction d
 d = np.zeros(n)
 d[j] = 1
 d[B] = la.solve(A_B, -A[:, j])
# Check for unboundedness
 if np.all(d >= 0):
     print("Unbounded problem!")
     return None, None, True
# Compute step length theta
 d_i = np.where(d[B] < 0)[0]
 theta = np.min(- x[B[d_i]] / d[B[d_i]])
 i = B[d_i[np.argmin(- x[B[d_i]] / d[B[d_i]])]]
# Compute next point
 x_next = x + theta * d
# Compute next basis
 B_next = B
 B_next[np.where(B == i)[0]] = j
 
 return x_next, B_next, -p, False

def simplex_algorithm(x, B, problem, max_iter=1000):
 """Run simplex algorithm"""
 for k in range(max_iter):
    x, B, y, end = simplex_iteration(x, B, problem)
    if end:
     break
 return x, B, y

In [33]:
M = 1000
problem = {
    'A': np.array([[5, 1, 3, 1, 0], 
                   [3, 1, 0, 0, 1]]),
    'b': np.array([8, 3]),
    'c': np.array([13, 10, 6, M, M])
}
A = np.array([[5, 1, 3, 1, 0]])
b = np.array([8, 3])
c = np.array([13, 10, 6, M, M])
              
x1 = np.array([0, 0, 0, 8, 3])
B =  np.array([3, 4]) 


x3, B3, y3 = simplex_algorithm(x1, B, problem, max_iter = 1000)
pvalue = c @ x3
dvalue = -b @ y3

print("Optimal Primal solution:",x3[:3])
print("Associated Basis:", B3)
print("Optimal Dual solution:", y3) 
print("Optimal Primal Value:", pvalue)
print("Optimal Dual Value:", dvalue)

    



Optimal solution found!
Optimal Primal solution: [1. 0. 1.]
Associated Basis: [2 0]
Optimal Dual solution: [-2. -1.]
Optimal Primal Value: 19.0
Optimal Dual Value: 18.999999999999996


In [23]:
x = cp.Variable(3)

#Define primal objective function(minimize)
objectivep = cp.Minimize(13 * x[0] + 10 * x[1] + 6 * x[2] )

constraintsp = [
    5 * x[0] + 1 * x[1] + 3 * x[2] == 8,
    3 * x[0] + x[1] == 3,
    x>=0
   
]

problem2 = cp.Problem(objectivep, constraintsp)
problem2.solve()
x_sol = x.value
print("x* :", x_sol)
print("Optimal primal value", problem2.value)

y = cp.Variable(2)
# Define dual objective function (maximize)
objective = cp.Maximize(-8 * y[0] - 3 * y[1])

# Define dual constraints
constraints = [
    5 * y[0] + 3 * y[1] + 13 >= 0,
    y[0] + y[1] +10 >= 0,
    3 * y[0] + 6 >= 0
]

prob = cp.Problem(objective, constraints)
prob.solve()
y_sol = y.value
print("y* :", y_sol)
print("optimal dual value:", prob.value)

#Context:
#The optimal solutions(both primal and dual) and values found in c are essentially the same 
# we found in a. 

x* : [ 1.00000000e+00 -3.60958917e-11  1.00000000e+00]
Optimal primal value 18.99999999974731
y* : [-2.         -1.00000001]
optimal dual value: 19.0000000006024
